In [1]:
import pandas as pd
import json
import re

In [4]:
# provide with the path to your JSON file
json_file_path = './Data/jira-conversations.json'

In [3]:

# Load the JSON file into a DataFrame
df = pd.read_json(json_file_path)

In [4]:
df.head()

,question000001,response000002,question000003,response000004,question000005,response000006,question000007,response000008,question000009,response000010,...,question000321,response000322,question000323,response000324,question000325,response000326,question000327,response000328,question000329,response000330
0,\n \nHii This is ~+~+~+~+~+~ gadipally UID-U6...,Thank you for reaching out. You are able to a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Hello, \n \n \nI am ~+~+~+~+~+~ with ~+~+~+~+...","Hello,\n~+~+~+~+~+~provide me with your USF em...","Hello, \n ~+~+~+~+~+~USF mail Id is [~~ema...",Canvas invite has been sent via email.,"Hello , \n I didn’t get any canvas invite ...",Invite has been sent again.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"\n \n \n \n~+~+~+~+~+~Sir/Madam,\n \n \n \n...",If you are interested in applying for another ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hi \nCould you kindly let me know my pre requi...,"Hi,\nThank you for your request to do a prereq...","Hi,\nThe system that I use to check your recor...",Hi \nI'm attaching my resume and unofficial...,"Hi, \nThank you for sending me the documents.\...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hi \nCould you kindly let me know my pre requi...,"Hi,\nThank you for your request to do a prereq...","Hi, \nThank you for your request to do a prere...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Load the JSON data from file
with open(json_file_path, 'r') as f:
    data = json.load(f)

In [18]:
new_data = []

def clean_text(text):
    # Remove sequences like '\n'
    text = text.replace('\\n', '')
    text = text.replace('/n', '')
    # Remove sequences like '~+~+~+~+~+~'
    text = re.sub(r'^~+\w+~+$', '', text)
    # Remove sequences like '[~~email~~-4675564125407667342~~]'
    text = re.sub(r'\[~~email~~-?\d+~~\]', ' ', text)
    text = re.sub(r'~\+~\+~\+~\+~\+|\n', ' ', text)
    # Removing words that start with 'U' followed by 7 digits
    text = re.sub(r'\bU\d{7}\b|\bU\d{8}\b', ' ', text)
    return text.strip()

for idx, d in enumerate(data, start=1):  # start=1 will start indexing from 1
    new_dict = {"Conversation": idx}  # Start the dictionary with the "Conversation" key
    queries = [key for key in d.keys() if "query" in key.lower()]
    responses = [key for key in d.keys() if "response" in key.lower()]

    for i in range(len(queries)):
        new_query_key = f"Student Query {i + 1}"
        new_dict[new_query_key] = clean_text(d[queries[i]])
        if i < len(responses):  # Ensure there's a matching response
            new_response_key = f"Response {i + 1} to the student"
            new_dict[new_response_key] = clean_text(d[responses[i]])

    new_data.append(new_dict)

# Saving the modified data with conversation numbers to the same file
with open('jira_conversation_cleaned.json', 'w') as f:
    json.dump(new_data, f, indent=4)

In [35]:
from fpdf import FPDF

In [38]:

def sanitize_text(text):
    # First, ensure the input is a string
    if not isinstance(text, str):
        text = str(text)
        
    # Replace smart single and double quotes
    text = text.replace('“', '"').replace('”', '"')
    text = text.replace('‘', "'").replace('’', "'")
    
    # Encode to latin1, ignoring characters that can't be encoded
    text = text.encode('latin1', 'ignore').decode('latin1')
    
    return text

# 1. Load the JSON file
with open("jira_conversation_cleaned.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# 2. Convert the JSON data into a readable string format
output_str = "Conversation Records of the Student and College Management\n\n"
for index, conversation in enumerate(data, 1):
    output_str += f"Conversation {index}:\n"
    for key, value in conversation.items():
        # Sanitize each value
        sanitized_value = sanitize_text(value)
        output_str += f"{key}: {sanitized_value}\n"
    output_str += "\n"  # Adding a newline for separation between conversations

# 3. Save the formatted string data to a PDF
class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Conversation Records", 0, 1, "C")

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 8)
        self.cell(0, 10, f"Page {self.page_no()}", 0, 0, "C")

pdf = PDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, output_str)
pdf.output("Jira_ConversationRecords.pdf")

''